    In this problem set, I want to examine the robustness of the most common version of GARCH model for stock return volatility. I plan to first find some most actively traded stocks on NASDAQ. Then I will scrape their daily prices from the most recent one year. subsequently, I will model their volatilities using GARCH model. Lastly, I analyze the model by checking the significance of the coefficients of the model.

In [5]:
% matplotlib inline
import pandas as pd
import pandas_datareader.data as web   
import datetime
import matplotlib as mlt
import numpy as np
from arch import arch_model
import urllib.request
from bs4 import BeautifulSoup

In [6]:
#get a list of most actively traded stocks on NASDAQ
active_stocks='http://www.nasdaq.com/markets/most-active.aspx'
header = {'User-Agent': 'Mozilla/5.0'}
req = urllib.request.Request(active_stocks, headers=header)
page = urllib.request.urlopen(req)
soup = BeautifulSoup(page, 'lxml')
table_area = soup.find(id='_active')
table=table_area.find('table')

In [33]:
ticker=[]
cells1=table.findAll('h3')
for row in cells1:
    ticker.append(row.find(text=True))



In [34]:
ticker

['ROKU',
 'MAT',
 'JD',
 'MU',
 'SIRI',
 'SGYP',
 'QCOM',
 'INTC',
 'NKTR',
 'AAPL',
 'CMCSA',
 'NVDA',
 'NVFY',
 'CSCO',
 'TVIX',
 'WBA',
 'QQQ',
 'MSFT',
 'ENDP',
 'XNET']

In [29]:
#set range of dates and read data from google finance
start = datetime.datetime(2017,8,14)
end = datetime.date.today()

In [47]:
stocks=pd.DataFrame()

for i in ticker:
    
    a = web.DataReader(i, "google", start, end)
    stocks[i]=(a["Close"])
    
stocks.head(5)

,ROKU,MAT,JD,MU,SIRI,SGYP,QCOM,INTC,NKTR,AAPL,CMCSA,NVDA,NVFY,CSCO,TVIX,WBA,QQQ,MSFT,ENDP,XNET
Date,,,,,,,,,,,,,,,,,,,,
2017-09-28,23.50,15.01,38.65,37.96,5.44,2.59,51.75,37.83,23.68,153.28,37.82,175.68,1.64,33.35,11.77,77.52,144.41,73.87,8.15,4.28
2017-09-29,26.54,15.48,38.20,39.33,5.52,2.90,51.84,38.08,24.00,154.12,38.48,178.77,1.66,33.63,11.33,77.22,145.45,74.49,8.56,4.25
2017-10-02,23.56,15.38,39.10,40.01,5.50,3.25,51.88,39.04,24.77,153.81,37.97,179.00,1.63,33.75,10.84,76.60,145.58,74.61,9.09,4.27
2017-10-03,20.81,15.68,39.21,40.37,5.60,3.23,52.02,39.38,24.50,154.48,38.30,179.37,1.61,33.85,10.76,76.79,145.89,74.26,8.71,4.40
2017-10-04,20.85,15.72,38.34,39.56,5.73,3.44,51.96,39.34,24.91,153.48,38.59,180.87,1.58,33.44,10.84,75.83,146.03,74.69,8.82,4.42


In [48]:
#Calculate stock log-return
stock_returns=stocks.apply(lambda x: np.log(x) - np.log(x.shift(1)))
stock_returns=stock_returns.dropna()

In [99]:
#apply GARCH model and calculate p-values for the coefficients of the model
result=pd.DataFrame({'Parameters\' p-values':['mu','omega','alpha','beta']})
for columns in stock_returns:
    a=arch_model(stock_returns[columns],p=1,q=1)
    b=a.fit(update_freq=10)
    result[columns]=b.pvalues.values
    

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -28.310991555716857
            Iterations: 7
            Function evaluations: 53
            Gradient evaluations: 7
Iteration:     10,   Func. Count:     73,   Neg. LLF: -61.459375273665245
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -61.53085709119814
            Iterations: 13
            Function evaluations: 92
            Gradient evaluations: 13
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -85.16034328578914
            Iterations: 7
            Function evaluations: 53
            Gradient evaluations: 7
Iteration:     10,   Func. Count:     70,   Neg. LLF: -80.31943062066298
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -80.31943062191628
            Iterations: 10
            Function evaluations: 81
            Gradient evaluations: 10
Optimization

In [108]:
print(result.to_latex())


\begin{tabular}{llrrrrrrrrrrrrrrrrrrrr}
\toprule
{} & Parameters' p-values &      ROKU &       MAT &            JD &        MU &      SIRI &      SGYP &      QCOM &      INTC &      NKTR &      AAPL &     CMCSA &      NVDA &      NVFY &      CSCO &          TVIX &       WBA &            QQQ &          MSFT &      ENDP &      XNET \\
\midrule
0 &                   mu &  0.510570 &  0.740742 &  0.000000e+00 &  0.112467 &  0.871762 &  0.477513 &  0.949363 &  0.000000 &  0.894868 &  0.000000 &  0.847885 &  0.020036 &  0.876005 &  0.000000 &  1.566342e-01 &  0.536953 &   0.000000e+00 &  0.000000e+00 &  0.487575 &  0.204933 \\
1 &                omega &  0.162054 &  0.133275 &  1.000000e+00 &  0.119935 &  0.006172 &  0.171380 &  0.191738 &  0.000000 &  0.374707 &  0.000000 &  0.003012 &  0.009344 &  0.196678 &  0.000000 &  3.270498e-01 &  0.140633 &   0.000000e+00 &  0.000000e+00 &  0.981363 &  0.609095 \\
2 &                alpha &  0.397775 &  0.154340 &  1.000000e+00 &  1.000000 &  1.0000

In [109]:
result

,Parameters' p-values,ROKU,MAT,JD,MU,SIRI,SGYP,QCOM,INTC,NKTR,...,CMCSA,NVDA,NVFY,CSCO,TVIX,WBA,QQQ,MSFT,ENDP,XNET
0,mu,0.510570,0.740742,0.000000e+00,0.112467,0.871762,0.477513,0.949363,0.000000,0.894868,...,0.847885,0.020036,0.876005,0.000000,1.566342e-01,0.536953,0.000000e+00,0.000000e+00,0.487575,0.204933
1,omega,0.162054,0.133275,1.000000e+00,0.119935,0.006172,0.171380,0.191738,0.000000,0.374707,...,0.003012,0.009344,0.196678,0.000000,3.270498e-01,0.140633,0.000000e+00,0.000000e+00,0.981363,0.609095
2,alpha,0.397775,0.154340,1.000000e+00,1.000000,1.000000,0.551914,0.285348,1.000000,0.203502,...,1.000000,0.205674,0.136573,0.999999,1.000000e+00,0.653389,9.998455e-01,1.000000e+00,1.000000,0.408368
3,beta,0.005137,1.000000,2.209084e-15,0.151965,1.000000,0.422063,0.003347,0.000166,0.046418,...,1.000000,0.163230,1.000000,0.000002,3.467206e-11,1.000000,1.071377e-100,4.420046e-96,0.880381,0.000779
